In [3]:
import numpy as np
import pandas as pd
%matplotlib inline

In [4]:
attacks = pd.read_csv('../assets/01_df.csv')

In [5]:
len(attacks)

1342

In [4]:
attacks

,Unnamed: 0,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,...,Time,Species,Investigator or Source,pdf,href formula,href,ts_year,ts_month,ts_day,ts_date
0,0,2017.11.30.b,30-Nov-2017,2017.0,Unprovoked,COSTA RICA,Cocos Island,Manuelita,Scuba Diving,__ Jimenez,...,NaN,"Tiger shark, female",R. Arauz,2017.11.30.b-Jimenez.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,30,2017-11-30
1,1,2017.11.30.a,30-Nov-2017,2017.0,Unprovoked,COSTA RICA,Cocos Island,Manuelita,Scuba diving,Rohina Bhandari,...,NaN,"Tiger shark, female",R. Arauz,2017.11.30.a-Bhandari.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,30,2017-11-30
2,2,2017.11.25.R,Reported 25-Nov-2017,2017.0,Sea Disaster,LIBYA,NaN,Gars Garabulli,2 boats capsized,31 migrants,...,NaN,Some drowned but other may have been killed by...,"TG Com 24, 11/25/2017",2017.11.25.R-Libya.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,25,2017-11-25
3,3,2017.11.24,24-Nov-2017,2017.0,Unprovoked,USA,California,"Stillwater Cove, Monterey County",Spearfishing,Grigor Azatian,...,13h45,"White shark, 4.6 m","R. Collier, GSAF",2017.11.24-Azatian.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,24,2017-11-24
4,4,2017.11.18,18-Nov-2017,2017.0,Unprovoked,USA,Florida,"Floridana Beach, Brevard County",Surfing,Kaia Anderson,...,Late afternoon,NaN,"Florida Today, 11/21/2017",2017.11.18-Anderson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,18,2017-11-18
5,5,2017.11.13.R,Reported 13-Nov-2017,2017.0,Unprovoked,NaN,NaN,NaN,Surfing,Timur Yunusov,...,NaN,NaN,Instagram,2017.11.13.R-Timur.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,13,2017-11-13
6,6,2017.11.13,13-Nov-2017,2017.0,Unprovoked,AUSTRALIA,New South Wales,Avoca Beach,Surfing,Charlie Fry,...,16h00,2 m shark,"B. Myatt, GSAF",2017.11.13-Fry.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,13,2017-11-13
7,7,2017.11.04,04-Nov-2017,2017.0,Unprovoked,CUBA,Holquin Province,Guardalavaca Beach,Night bathing,Jesús Cabrera González,...,23h30,Tiger shark,TrackingSharks.com,2017.11.04-Cuba.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,4,2017-11-04
8,8,2017.10.31.R,Reported 31-Oct-2017,2017.0,Unprovoked,AUSTRALIA,Queensland,Magnetic Island,Snorkeling,Jack Tolley,...,NaN,Cookiecutter shark,"Townsville Bulletin, 10/31/2017",2017.10.21.R-Tolley.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,10,31,2017-10-31
9,9,2017.10.28,28-Oct-2017,2017.0,Unprovoked,AUSTRALIA,New South Wales,Birubi Point,Surfing,male,...,09h00,"Wobbegong shark, 1 m","B. Myatt, GSAF",2017.10.28-Birubi.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,10,28,2017-10-28


In [5]:
attacks['href formula'][1]

'http://sharkattackfile.net/spreadsheets/pdf_directory/2017.11.30.a-Bhandari.pdf'

In [11]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

In [7]:
attacks['loc_str'] = attacks['Location']+" " +attacks['Area']+" "+attacks['Country']

In [9]:
len(attacks['loc_str'].unique())

957

In [13]:
#scrape long/lat
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 7)
coords_3 = []
driver.get('https://www.google.com/maps')
for location in attacks['loc_str'].unique():
    loca = str(location)
    loca = loca.strip(' \t\n\r')
    if (loca == "NaN") | (loca == "nan"):
        next
    else:
        searchbox = wait.until(EC.presence_of_element_located((By.ID, 'searchboxinput')))
        searchbox.clear()
        searchbox.send_keys(loca)

        driver.find_element_by_id('searchbox-searchbutton').click()
        sleep(6)
        url = driver.current_url
        tries = 0
        while '@' not in url:
            tries += 1
            sleep(1)
            url = driver.current_url
            if tries == 5: #try five times to give the url the chance to resolve
                break
        try:
            long_lat = url.split('@')[1].split(',')[:2]
            coords_3.append((loca, long_lat[0], long_lat[1]))
        except:
            coords_3.append((loca, None, None)) 
driver.quit()  

In [14]:
coords_3 = pd.DataFrame(coords_3)

In [15]:
coords_3

,0,1,2
0,Manuelita Cocos Island COSTA RICA,5.5313198,-87.0961449
1,"Stillwater Cove, Monterey County California USA",36.5639138,-121.9445819
2,"Floridana Beach, Brevard County Florida USA",27.9448942,-80.505628
3,Avoca Beach New South Wales AUSTRALIA,-33.4705196,151.4107836
4,Guardalavaca Beach Holquin Province CUBA,21.1162344,-75.879118
5,Magnetic Island Queensland AUSTRALIA,-19.1421512,146.7963652
6,Birubi Point New South Wales AUSTRALIA,-32.7884083,152.0715595
7,"White Plains Beach, Oahu Hawaii USA",21.3030285,-158.0468662
8,Le Goulet Pamplemousses MAURITIUS,-20.1044023,57.5155488
9,"Juno Beach, Palm Beach County Florida USA",26.875057,-80.0751825


In [61]:
coords = pd.DataFrame(coords)

In [16]:
coords_3.columns = ['loc_str', "Long", "Lat"]

In [18]:
coords_3.to_csv('attack_coords_2.csv')

In [61]:
coords= pd.read_csv('../assets/attack_coords.csv')

In [62]:
coords = coords.drop("Unnamed: 0", axis =1)

In [20]:
attacks = attacks.merge(coords_3, on = "loc_str", how = 'left')

In [24]:
attacks

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,pdf,href formula,href,ts_year,ts_month,ts_day,ts_date,loc_str,Long,Lat
0,2017.11.30.b,30-Nov-2017,2017.0,Unprovoked,COSTA RICA,Cocos Island,Manuelita,Scuba Diving,__ Jimenez,M,...,2017.11.30.b-Jimenez.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,30,2017-11-30,Manuelita Cocos Island COSTA RICA,5.5313198,-87.0961449
1,2017.11.30.a,30-Nov-2017,2017.0,Unprovoked,COSTA RICA,Cocos Island,Manuelita,Scuba diving,Rohina Bhandari,F,...,2017.11.30.a-Bhandari.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,30,2017-11-30,Manuelita Cocos Island COSTA RICA,5.5313198,-87.0961449
2,2017.11.25.R,Reported 25-Nov-2017,2017.0,Sea Disaster,LIBYA,NaN,Gars Garabulli,2 boats capsized,31 migrants,NaN,...,2017.11.25.R-Libya.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,25,2017-11-25,NaN,NaN,NaN
3,2017.11.24,24-Nov-2017,2017.0,Unprovoked,USA,California,"Stillwater Cove, Monterey County",Spearfishing,Grigor Azatian,M,...,2017.11.24-Azatian.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,24,2017-11-24,"Stillwater Cove, Monterey County California USA",36.5639138,-121.9445819
4,2017.11.18,18-Nov-2017,2017.0,Unprovoked,USA,Florida,"Floridana Beach, Brevard County",Surfing,Kaia Anderson,F,...,2017.11.18-Anderson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,18,2017-11-18,"Floridana Beach, Brevard County Florida USA",27.9448942,-80.505628
5,2017.11.13.R,Reported 13-Nov-2017,2017.0,Unprovoked,NaN,NaN,NaN,Surfing,Timur Yunusov,M,...,2017.11.13.R-Timur.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,13,2017-11-13,NaN,NaN,NaN
6,2017.11.13,13-Nov-2017,2017.0,Unprovoked,AUSTRALIA,New South Wales,Avoca Beach,Surfing,Charlie Fry,M,...,2017.11.13-Fry.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,13,2017-11-13,Avoca Beach New South Wales AUSTRALIA,-33.4705196,151.4107836
7,2017.11.04,04-Nov-2017,2017.0,Unprovoked,CUBA,Holquin Province,Guardalavaca Beach,Night bathing,Jesús Cabrera González,M,...,2017.11.04-Cuba.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,11,4,2017-11-04,Guardalavaca Beach Holquin Province CUBA,21.1162344,-75.879118
8,2017.10.31.R,Reported 31-Oct-2017,2017.0,Unprovoked,AUSTRALIA,Queensland,Magnetic Island,Snorkeling,Jack Tolley,M,...,2017.10.21.R-Tolley.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,10,31,2017-10-31,Magnetic Island Queensland AUSTRALIA,-19.1421512,146.7963652
9,2017.10.28,28-Oct-2017,2017.0,Unprovoked,AUSTRALIA,New South Wales,Birubi Point,Surfing,male,M,...,2017.10.28-Birubi.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017,10,28,2017-10-28,Birubi Point New South Wales AUSTRALIA,-32.7884083,152.0715595


In [23]:
attacks = attacks.drop('Unnamed: 0', axis =1)

In [ ]:
#some of these are inaccurate. This might be a waste of time?

In [ ]:
for href in attacks['href formula']:
    
    

In [27]:
fp = open(attacks['href formula'][1], 'rb')
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine

parser = PDFParser(fp)
doc = PDFDocument()
parser.set_document(doc)
doc.set_parser(parser)
doc.initialize('')
rsrcmgr = PDFResourceManager()
laparams = LAParams()
laparams.char_margin = 1.0
laparams.word_margin = 1.0
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
extracted_text = ''

for page in doc.get_pages():
    interpreter.process_page(page)
    layout = device.get_result()
    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            extracted_text += lt_obj.get_text()

NameError: name 'PdfFileWriter' is not defined

In [22]:
extracted_text

'This isaSamplePDF\nThankyou forselectingtheoptiontoreceiveyourstatement imageelectronically.You\nwillnoticethere isaSecurityCodedisplayedwithinthisSamplePDF.Thiscode,asseen\nbelow,mustbeenteredcorrectly intheconfirmationboxpresentedontheStatement\nPreferencescreen.Inordertocompletethisrequest,youmustclicktheIAGREEbutton\nafterenteringthecode.\nIfyouareunabletoreadtheSecurityCode,simplyclosethisSamplePDFandclickthe\nlinkagaintoreceiveanewcode.\nByopeningandviewingthissamplePDF imageandobtainingyourSecurityCode,you\nhavedemonstratedthatyourdevicesatisfiesthehardwareandsoftwarerequirements\nexplainedbelowandwillbeabletoretrieveyourstatementselectronicallynowand inthe\nfuture.\nValidationCode\n39fe5a4\n'

In [16]:
import pip
installed_packages = pip.get_installed_distributions()
installed_packages_list = sorted(["%s==%s" % (i.key, i.version)
     for i in installed_packages])
print(installed_packages_list)

['alabaster==0.7.9', 'anaconda-clean==1.0', 'anaconda-client==1.5.1', 'anaconda-navigator==1.3.1', 'appnope==0.1.0', 'appscript==1.0.1', 'arcgis==1.2.4', 'argcomplete==1.0.0', 'astroid==1.4.7', 'astropy==2.0.2', 'awscli==1.11.66', 'babel==2.3.4', 'backports.shutil-get-terminal-size==1.0.0', 'basemap==1.0.7', 'beautifulsoup4==4.5.1', 'bitarray==0.8.1', 'blaze==0.10.1', 'blinker==1.4', 'bokeh==0.12.2', 'boto3==1.4.8', 'boto==2.42.0', 'botocore==1.8.5', 'bottleneck==1.2.1', 'brewer2mpl==1.4.1', 'bs4==0.0.1', 'bz2file==0.98', 'certifi==2017.7.27.1', 'cffi==1.7.0', 'chardet==3.0.4', 'chest==0.2.3', 'click-plugins==1.0.3', 'click==6.7', 'cligj==0.4.0', 'cloudpickle==0.2.1', 'clyent==1.2.2', 'colorama==0.3.7', 'conda-build==2.0.2', 'conda==4.3.30', 'configobj==5.0.6', 'contextlib2==0.5.3', 'cryptography==1.5', 'cycler==0.10.0', 'cymem==1.31.2', 'cython==0.24.1', 'cytoolz==0.8.0', 'dask==0.11.0', 'datashape==0.5.2', 'decorator==4.1.2', 'descartes==1.1.0', 'dill==0.2.5', 'documentfeatureselecti